In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
source_path = 'https://www.hackerearth.com/challenges/?filters=competitive%2Chackathon%2Chiring%2Cuniversity'
page = requests.get(source_path)
page_content = page.content

In [3]:
soup = BeautifulSoup(page_content, 'html.parser')
print (soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">
<html xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="#323754" name="theme-color"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="

    Participate in monthly programming challenges, coding competitions, and hackathons. Register now to compete with top developers worldwide, and win exciting prizes!

" name="description"/>
  <link href="https://www.hackerearth.com/challenges/" rel="canonical"/>
  <meta content="1BF418E510B5A17A2D71685C571BF4EB" name="msvalidate.01">
   <meta content="width=device-width,initial-scale=1.0" name="viewport"/>
   <meta content="strict-origin-when-cross-origin" name="referrer"/>
   <script>
    (function() {
              if (window.BOOMR && window.BOOMR.version) {
                  return;
              }
              var dom, doc, where, iframe = document.createElement("iframe"),
          

In [4]:
import re
def starts_with_special_char(string):
    if re.match(r'^[^\w]', string):
        return True
    return False

results_list = soup.find_all('span', class_='challenge-list-title')
# get participants outside of <i class="fa fa-users"></i>
participants = soup.find_all('div', class_='registrations')
total_participants_list = []
for i in participants:
    total_participants_list.append(i.text.strip())
old_participants = soup.find_all('span', class_='reg')
for i in old_participants:
    total_participants_list.append(i.text.strip())

print(total_participants_list)

# Let's see what the search results look like
hackathon_list = []
for i in range(len(results_list)):
    if not starts_with_special_char(results_list[i].text.strip()):
        temp_dic = {}
        temp_dic['hackathon'] = results_list[i].text.strip()
        temp_dic['participants'] = int(total_participants_list[i])
        hackathon_list.append(temp_dic)
hackathon_list

['929', '14259', '42', '48409', '959', '965', '48355', '1626', '1357', '7587', '3139', '46763', '835', '48555']


[{'hackathon': 'Build a Climate Fintech App', 'participants': 929},
 {'hackathon': 'IBM zStudent Contest', 'participants': 14259},
 {'hackathon': 'Buidl the Future: A Global Hack for Web3', 'participants': 42},
 {'hackathon': "October Circuits '22", 'participants': 48409},
 {'hackathon': 'DSA Coding Contest - October 22', 'participants': 959},
 {'hackathon': 'All Things e-Commerce - GeTS Hackathon', 'participants': 965},
 {'hackathon': 'October Easy 22', 'participants': 48355},
 {'hackathon': 'Quantum Science and Technology Hackathon 2022',
  'participants': 1626},
 {'hackathon': "Build With Celo ReFi Hackathon '22", 'participants': 1357},
 {'hackathon': 'Ballerina Coding Challenge', 'participants': 7587},
 {'hackathon': "September Circuits '22", 'participants': 46763},
 {'hackathon': 'Data Structures and Algorithms Coding Contest - September 22',
  'participants': 835},
 {'hackathon': 'September Easy 22', 'participants': 48555}]

In [ ]:
df = pd.DataFrame(hackathon_list)
df.to_csv('./hackerearth_topics.csv')

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")

In [ ]:
es.indices.delete(index='hackerearth', ignore=[400, 404])
# create a new index with mappings
es.indices.create(index='hackerearth', body={
    "mappings": {
        "properties": {
            "hackathon":{
                "type":"text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                    }
                }
            },
            "participants":{
                "type":"integer"
            }
        }
    }
})
# insert keywordbags into elasticsearch
elastic_id = 1
for i in hackathon_list:
    if(i['hackathon'] != '' or not starts_with_special_char(i['hackathon'])):
        print(i['hackathon'])
        es.index(index='hackerearth', id=elastic_id, body=i)
        elastic_id += 1